In [1]:
import cv2
import mediapipe as mp
import time

class FaceDetector():
    def __init__(self,minDetectionCon=0.5):
         self.mpFaceDetection=mp.solutions.face_detection
         self.mpDraw=mp.solutions.drawing_utils
         self.minDetectionCon=minDetectionCon
         self.faceDetection=self.mpFaceDetection.FaceDetection(min_detection_confidence=self.minDetectionCon)
    def findFaces(self,img,draw=True):
        bboxs=[]
        imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        results=self.faceDetection.process(imgRGB)
        if results.detections:
            for id,detection in enumerate(results.detections):
                bbox=detection.location_data.relative_bounding_box
                ih,iw,ic=img.shape
                x,y,w,h=int(bbox.xmin*iw),int(bbox.ymin*ih),int(bbox.width*iw),int(bbox.height*ih)
                check=int(detection.score[0]*100)
                bboxs.append([id,(x,y,h,w),check])
                if draw and check>=60:
                    img=self.fancyDraw(img,(x,y,h,w))
                    cv2.putText(img,f'{int(detection.score[0]*100)}%',(x,y-20),cv2.FONT_HERSHEY_PLAIN,2,(0,255,0),2)
        return img,bboxs
    def fancyDraw(self,img,bbox,l=30,t=4):
        x,y,h,w=bbox
        x1,y1=x+w,y+h
        cv2.rectangle(img,(x,y),(x+h,y+w),(255,0,255),2)
        # upper left
        cv2.line(img,(x,y),(x,y+l),(0,255,0),t)
        cv2.line(img,(x,y),(x+l,y),(0,255,0),t)
        
        # lower right
        cv2.line(img,(x1,y1),(x1,y1-l),(0,255,0),t)
        cv2.line(img,(x1,y1),(x1-l,y1),(0,255,0),t)
        
        # upper right
        cv2.line(img,(x1,y),(x1,y+l),(0,255,0),t)
        cv2.line(img,(x1,y),(x1-l,y),(0,255,0),t)
        
        # lower left
        cv2.line(img,(x,y1),(x,y1-l),(0,255,0),t)
        cv2.line(img,(x,y1),(x+l,y1),(0,255,0),t)
        return img
        
    
def main():
    cap=cv2.VideoCapture(0)
    detector=FaceDetector()
    iTime=0
    pTime=0
    while True:
        success,img=cap.read()
        img,bboxs=detector.findFaces(img)
        
        iTime=time.time()
        fps=1/(iTime-pTime)
        pTime=iTime
        cv2.putText(img,str(int(fps)),(20,70),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),5)
        cv2.imshow("img",img)
        if cv2.waitKey(1) & 0xFF==ord('a'):
            break
    cap.release()
    cv2.destroyAllWindows()
if __name__=='__main__':
    main()
    